In [ ]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import OxfordIIITPet, Flowers102
import scipy

import copy
import numpy as np
import re

## Load Dataset

In [ ]:
class NeuralNetwork():
    def __init__(self, basemodel, output, train_loader, val_loader, test_loader, unfreeze=0, parameters=None):
        # Init model
        self.model = copy.deepcopy(basemodel)
        # changing the fully connected layer
        self.model.fc = nn.Linear(self.model.fc.in_features, output)
        # Unfreeze the L last layers and fully connected last layer
        self.unfreeze_layers(unfreeze)
        self.model = self.model.to(device)

        self.optimizer = optim.Adam([{"params": self.model.fc.parameters(),"lr": 1e-3, "weight_decay": 1e-4},])
        self.criterion = nn.CrossEntropyLoss()

        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader

        self.parameters = parameters if parameters is not None else {}


    def unfreeze_layers(self, L):
      # freezing all layers
      for p in self.model.parameters():
          p.requires_grad = False

      layer = 0
      # unfreeze last L layers and fully connected layer
      for name, p in reversed(list(self.model.named_parameters())):
        if layer < L+1:
          if not name.endswith("bias"):
            layer += 1
          p.requires_grad = True

    def check_trainable_layers(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                print(f"{name} requires grad")
            else:
                print(f"{name} does NOT require grad")


    def run_epoch(self, loader, train=True):
        self.model.train() if train else self.model.eval()
        total_loss, correct, total = 0.0, 0, 0
        for imgs, labs in loader:
            imgs, labs = imgs.to(device), labs.to(device)
            if train:
                self.optimizer.zero_grad()
            logits = self.model(imgs)
            loss   = self.criterion(logits, labs)
            if train:
                loss.backward()
                self.optimizer.step()
            total_loss += loss.item() * imgs.size(0)
            preds      = logits.argmax(dim=1)
            correct   += (preds == labs).sum().item()
            total     += imgs.size(0)
        return total_loss/total, correct/total*100


    def test_model(self, test_loader=None):
      if test_loader != None:
        loader = test_loader
      else:
        loader = self.test_loader

      test_loss, test_acc = self.run_epoch(loader, train=False)
      print(f"Test Accuracy: {test_acc:.2f}%")
      return test_acc

    def train(self, epochs, path="model.pth"):
      best_val_acc = 0.0
      for epoch in range(1,epochs+1):

        # gradually unfreeze layers for strategy 2 according to rate parameter
        if 'gradual_unfreezing' in self.parameters:
            uf_rate = 1 if not 'unfreezing_rate' in self.parameters else parameters['unfreezing_rate']
            L = np.floor(uf_rate*(1+epoch)) # allow for rates < 1
            if L < 1:
                L = 1
            self.unfreeze_layers(L)

        tr_loss, tr_acc = self.run_epoch(self.train_loader, train=True)
        val_loss, val_acc = self.run_epoch(self.val_loader, train=False)
        print(f"Epoch {epoch:2d} | Train: {tr_acc:.2f}% | Val: {val_acc:.2f}%")
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(self.model.state_dict(), path)

      test_acc = self.test_model()
      return test_acc

    def train2(self, epochs, path="model.pth", tee = False, test_loader=None): # tee=test_eac_epoch
        best_val_acc = 0.0
        for epoch in range(1, epochs+1):
            # Gradual unfreezing
            if self.parameters.get('gradual_unfreezing', False):
                uf_rate = self.parameters.get('unfreezing_rate', 1)
                L = int(np.floor(uf_rate * (epoch + 1)))
                if L < 1:
                    L = 1
                self.unfreeze_layers(L)

            tr_loss, tr_acc = self.run_epoch(self.train_loader, train=True)
            val_loss, val_acc = self.run_epoch(self.val_loader, train=False)
            print(f"Epoch {epoch:2d} | Train: {tr_acc:.2f}% | Val: {val_acc:.2f}%")

            # Save best
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(self.model.state_dict(), path)

            # Step scheduler if present
            if hasattr(self, 'scheduler') and self.scheduler is not None:
                self.scheduler.step()

        return self.test_model()

    def train_catastrophic(self, epochs, path="model.pth", tee = True, test_loader=None): # tee=test_eac_epoch
        best_val_acc = 0.0
        results = {"primary": [], "secondary":[]}
        for epoch in range(1, epochs+1):
            # Gradual unfreezing
            if self.parameters.get('gradual_unfreezing', False):
                uf_rate = self.parameters.get('unfreezing_rate', 1)
                L = int(np.floor(uf_rate * (epoch + 1)))
                if L < 1:
                    L = 1
                self.unfreeze_layers(L)

            tr_loss, tr_acc = self.run_epoch(self.train_loader, train=True)
            val_loss, val_acc = self.run_epoch(self.val_loader, train=False)
            print(f"Epoch {epoch:2d} | Train: {tr_acc:.2f}% | Val: {val_acc:.2f}%")

            # Save best
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(self.model.state_dict(), path)

            if tee:
                results["primary"].append(self.test_model())
                if test_loader != None:
                    results["secondary"].append(self.test_model(test_loader = test_loader))
            # Step scheduler if present
            if hasattr(self, 'scheduler') and self.scheduler is not None:
                self.scheduler.step()
        if not tee:
            return self.test_model()

        return results

    def evaluate(self):
        self.model.eval()
        correct = 0
        total = 0

        # Track per-class accuracy
        class_correct = [0] * len(breed_names)
        class_total = [0] * len(breed_names)

        with torch.no_grad():
            for inputs, targets in self.test_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                outputs = self.model(inputs)
                _, predicted = outputs.max(1)

                # Overall accuracy
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                # Per-class accuracy
                for i in range(len(targets)):
                    label = targets[i].item()
                    class_total[label] += 1
                    if predicted[i] == targets[i]:
                        class_correct[label] += 1

        overall_acc = 100.0 * correct / total
        per_class_acc = [(100.0 * class_correct[i] / class_total[i] if class_total[i] > 0 else 0.0)
                          for i in range(len(breed_names))]

        return overall_acc, per_class_acc, class_total


In [ ]:
OxfordIIITPet(root="data", download=True)

100%|██████████| 792M/792M [00:34<00:00, 22.9MB/s]
100%|██████████| 19.2M/19.2M [00:01<00:00, 11.4MB/s]


Dataset OxfordIIITPet
    Number of datapoints: 3680
    Root location: data

In [ ]:
annnotations_dir = os.path.join("data", "oxford-iiit-pet", "annotations")
lines_for_files = open(os.path.join(annnotations_dir, "list.txt")).read().splitlines()[6:]
species_map = {l.split()[0]: int(l.split()[2]) for l in lines_for_files}

In [ ]:
with open(os.path.join(annnotations_dir, "trainval.txt")) as f:
    trainval_names = [l.split()[0] for l in f if l.strip()]
with open(os.path.join(annnotations_dir, "test.txt")) as f:
    test_names = [l.split()[0] for l in f if l.strip()]

In [ ]:
class BinaryPet(Dataset):
    def __init__(self, root, names, species_map, transform=None):
        self.root = root
        self.names = names
        self.species_map = species_map
        self.transform = transform

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        name = self.names[idx]
        img_path = os.path.join(
            self.root, "oxford-iiit-pet", "images", name + ".jpg"
        )
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.species_map[name] - 1  # 1→cat→0, 2→dog→1
        return img, label

In [ ]:
IMG_SIZE = 224

In [ ]:
transformIMG = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

In [ ]:
full = BinaryPet("data", trainval_names, species_map, transform=transformIMG)
n_val = int(0.1 * len(full))
train_ds, val_ds = random_split(full, [len(full)-n_val, n_val])

In [ ]:
test_ds = BinaryPet("data", test_names, species_map, transform=transformIMG)

In [ ]:
batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Binary Classification

## Model Set Up

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
# freezing all
for p in model.parameters():
    p.requires_grad = False
# changing the fully connected layer
model.fc = nn.Linear(model.fc.in_features, 2)
# Unfreeze the fully connected last layer
for name, p in model.named_parameters():
    if name.startswith("fc"):
        p.requires_grad = True
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 149MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam([
    {"params": model.fc.parameters(),     "lr": 1e-3, "weight_decay": 1e-4},
])

In [ ]:
print(torch.cuda.is_available())

True


## Training

In [ ]:
def run_epoch(model, loader, train=True, criterion=criterion, optimizer=optimizer): # TODO: Include criterion and optimizer in a neater way
    model.train() if train else model.eval()
    total_loss, correct, total = 0.0, 0, 0
    for imgs, labs in loader:
        imgs, labs = imgs.to(device), labs.to(device)
        if train:
            optimizer.zero_grad()
        logits = model(imgs)
        loss   = criterion(logits, labs)
        if train:
            loss.backward()
            optimizer.step()
        total_loss += loss.item() * imgs.size(0)
        preds      = logits.argmax(dim=1)
        correct   += (preds == labs).sum().item()
        total     += imgs.size(0)
    return total_loss/total, correct/total*100


In [ ]:
best_val_acc = 0.0
for epoch in range(1, 25):
    tr_loss, tr_acc = run_epoch(model, train_loader, train=True)
    val_loss, val_acc = run_epoch(model, val_loader, train=False)
    print(f"Epoch {epoch:2d} | Train: {tr_acc:.2f}% | Val: {val_acc:.2f}%")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_binary_resnet34.pth")

/Users/youngbinpyo/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/youngbinpyo/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/youngbinpyo/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'BinaryPet' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load("best_binary_resnet34.pth"))
test_loss, test_acc = run_epoch(model, test_loader, train=False)
print(f"Test Accuracy: {test_acc:.2f}%")

FileNotFoundError: [Errno 2] No such file or directory: 'best_binary_resnet34.pth'

# Multi Class Classification

TODO:
* different learning rates/learning rate schedulers for different layers
* data augmentation
* Effect of fine-tuning batch-norm parameters
* Train with Imbalanced Dataset
* Testing and logging of the above
* gradual_unfreezing testing
* unfreezing l layers at a time testing
  
DONE:
* gradual_unfreezing
* unfreezing l layers at a time

## Load Dataset

In [ ]:
annnotations_dir = os.path.join("data", "oxford-iiit-pet", "annotations")
lines_for_files = open(os.path.join(annnotations_dir, "list.txt")).read().splitlines()[6:]
breeds_map = {l.split()[0]:re.split(r'_\d+', l.split()[0])[0] for l in lines_for_files}

In [ ]:
breed_to_id = {val: i for i, val in enumerate(sorted(set(breeds_map.values())))}
id_to_breed = {v: k for k, v in breed_to_id.items()}

In [ ]:
class MultiClassPet(Dataset):
    def __init__(self, root, names, breeds_map, transform=None):
        self.root = root
        self.names = names
        self.breeds_map = breeds_map
        self.transform = transform

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        name = self.names[idx]
        img_path = os.path.join(
            self.root, "oxford-iiit-pet", "images", name + ".jpg"
        )
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = breed_to_id[self.breeds_map[name]]
        return img, label

In [ ]:
full = MultiClassPet("data", trainval_names, breeds_map, transform=transformIMG)
n_val = int(0.1 * len(full))
train_ds, val_ds = random_split(full, [len(full)-n_val, n_val])
test_ds = MultiClassPet("data", test_names, breeds_map, transform=transformIMG)


In [ ]:
batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

## Model Set Up

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
basemodel = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 193MB/s]


-> Strategy 1

In [ ]:
# Strategy 1: train with l layers unfrozen simultaneously
accs = []
for L in range(1, 10):
  print(f"Start training model with {L} last layers trainable")
  modelmulti = NeuralNetwork(basemodel, 37, train_loader, val_loader, test_loader, unfreeze=L)
  accs.append(modelmulti.train(10))

Start training model with 1 last layers trainable


KeyboardInterrupt: 

In [ ]:


# Transforms
base_tf = transforms.Compose([
    transforms.Resize(256), transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
aug_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(), transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# Create datasets and split 90/10
full_ds = MultiClassPet("data", trainval_names, breeds_map, transform=base_tf)
n_val   = int(0.1 * len(full_ds))
train_ds, val_ds = random_split(full_ds, [len(full_ds)-n_val, n_val])
test_ds = MultiClassPet("data", test_names, breeds_map, transform=base_tf)

# DataLoaders
batch_size = 32
val_loader  = DataLoader(val_ds,  batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Strategy 1 grid
results = []
basemodel = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)

for use_aug in [False, True]:
    for freeze_bn in [False, True]:
        for use_l2 in [False, True]:
            for use_sched in [False, True]:
                print(f"\n>>> aug={use_aug}, freeze_bn={freeze_bn}, L2={use_l2}, sched={use_sched}")

                # a) Set train transform
                train_ds.dataset.transform = aug_tf if use_aug else base_tf

                # b) Train loader
                train_loader = DataLoader(
                    train_ds, batch_size=batch_size, shuffle=True,
                    num_workers=4, pin_memory=True
                )

                # c) Init model, fine-tune last 6 layers
                model = NeuralNetwork(
                    basemodel=basemodel,
                    output=len(breed_names),
                    train_loader=train_loader,
                    val_loader=val_loader,
                    test_loader=test_loader,
                    unfreeze=6,
                    parameters={}
                )
                model.model.to(device)

                # d) Freeze batch-norm if requested
                if freeze_bn:
                    for m in model.model.modules():
                        if isinstance(m, nn.BatchNorm2d):
                            m.eval()
                            for p in m.parameters():
                                p.requires_grad = False

                # e) Optimizer with two LR groups, optional L2 weight decay
                wd = 1e-2 if use_l2 else 0.0
                head_p = list(model.model.fc.parameters())
                back_p = [p for n,p in model.model.named_parameters()
                          if p.requires_grad and not n.startswith("fc")]
                model.optimizer = optim.AdamW([
                    {"params": head_p, "lr":1e-3, "weight_decay":wd},
                    {"params": back_p, "lr":1e-4, "weight_decay":wd}
                ])

                # f) Scheduler if requested
                if use_sched:
                    model.scheduler = optim.lr_scheduler.StepLR(model.optimizer, step_size=5, gamma=0.1)
                else:
                    model.scheduler = None

                # g) Train and record test accuracy
                acc = model.train(epochs=10)
                results.append(((use_aug, freeze_bn, use_l2, use_sched), acc))

# Summary for comparison
print("\n=== Strategy 1 Extended Grid Results ===")
for (aug, bn, l2, sched), acc in results:
    print(f"aug={aug}, freeze_bn={bn}, L2={l2}, sched={sched} → Test Acc: {acc:.2f}%")



>>> aug=False, freeze_bn=False, L2=False, sched=False
Epoch  1 | Train: 74.61% | Val: 86.14%
Epoch  2 | Train: 94.11% | Val: 89.95%
Epoch  3 | Train: 97.49% | Val: 89.40%
Epoch  4 | Train: 99.49% | Val: 89.67%
Epoch  5 | Train: 99.61% | Val: 89.95%
Epoch  6 | Train: 99.64% | Val: 90.49%
Epoch  7 | Train: 99.88% | Val: 90.22%
Epoch  8 | Train: 99.94% | Val: 89.95%
Epoch  9 | Train: 99.82% | Val: 89.13%
Epoch 10 | Train: 99.91% | Val: 89.40%
Test Accuracy: 89.62%

>>> aug=False, freeze_bn=False, L2=False, sched=True
Epoch  1 | Train: 75.91% | Val: 88.86%
Epoch  2 | Train: 94.75% | Val: 88.59%
Epoch  3 | Train: 97.83% | Val: 88.32%
Epoch  4 | Train: 99.12% | Val: 89.67%
Epoch  5 | Train: 99.55% | Val: 89.67%
Epoch  6 | Train: 99.85% | Val: 90.22%
Epoch  7 | Train: 99.91% | Val: 89.40%
Epoch  8 | Train: 99.97% | Val: 89.95%
Epoch  9 | Train: 99.97% | Val: 89.95%
Epoch 10 | Train: 99.97% | Val: 90.22%
Test Accuracy: 90.87%

>>> aug=False, freeze_bn=False, L2=True, sched=False
Epoch  1 | Tr

-> Strategy 2

In [ ]:
# Strategy 2: train with gradual unfreezing
parameters = {}
parameters['gradual_unfreezing'] = True
accs = []

# test rates in steps of two
for rate in [0.3, 0.5, 0.8, 1, 2, 3]:
  print(f"Start training model with uf_rate {rate}")
  parameters['unfreezing_rate'] = rate
  modelmulti = NeuralNetwork(basemodel, 37, train_loader, val_loader, test_loader, unfreeze=1, parameters=parameters)   # start with only one unfrozen layer
  accs.append(modelmulti.train(10))

Start training model with uf_rate 0.3
Epoch  1 | Train: 61.08% | Val: 83.97%
Epoch  2 | Train: 88.68% | Val: 89.13%
Epoch  3 | Train: 91.46% | Val: 90.76%
Epoch  4 | Train: 92.39% | Val: 90.22%
Epoch  5 | Train: 93.75% | Val: 91.30%
Epoch  6 | Train: 94.93% | Val: 90.22%
Epoch  7 | Train: 95.77% | Val: 91.03%
Epoch  8 | Train: 95.92% | Val: 90.22%
Epoch  9 | Train: 96.71% | Val: 89.95%
Epoch 10 | Train: 97.04% | Val: 90.22%
Test Accuracy: 90.27%
Start training model with uf_rate 0.5
Epoch  1 | Train: 60.60% | Val: 83.15%
Epoch  2 | Train: 88.07% | Val: 87.23%
Epoch  3 | Train: 91.70% | Val: 89.67%
Epoch  4 | Train: 93.48% | Val: 89.95%
Epoch  5 | Train: 93.72% | Val: 90.49%
Epoch  6 | Train: 94.96% | Val: 89.95%
Epoch  7 | Train: 95.59% | Val: 90.49%
Epoch  8 | Train: 95.59% | Val: 89.40%
Epoch  9 | Train: 95.98% | Val: 89.13%
Epoch 10 | Train: 96.68% | Val: 89.40%
Test Accuracy: 89.07%
Start training model with uf_rate 0.8
Epoch  1 | Train: 59.48% | Val: 85.60%
Epoch  2 | Train: 88.77

In [ ]:


# transforms
base_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225]),
])
auga = [ transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224, scale=(k,1.0)),
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225]),
]) for k in [0.5, 0.6, 0.8 ]]

augb = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225]),
])

augc = [ transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    #transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(k),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225]),
]) for k in [10, 15, 25, 40]]

# model
basemodel = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)

# fixed unfreeze rate
fixed_rate = 2.0

# grid definitions
augmentations = [True]
freeze_bns    = [False]
use_l2s       = [False]
schedulers    = ['none']
augies = [base_tf]
# augies += auga
# augies += [augb]
augies = augc
augies += [augb]


results = []

# running the grid
for use_aug in augmentations:
    for freeze_bn in freeze_bns:
        for use_l2 in use_l2s:
            for sched in schedulers:
              for aug in augies:
                  # update train transform
                  train_ds.dataset.transform = aug if use_aug else base_tf
                  train_loader = DataLoader(
                      train_ds,
                      batch_size=32,
                      shuffle=True,
                      num_workers=4,
                      pin_memory=True
                  )
                  print(f"\n>>> aug={use_aug}, freeze_bn={freeze_bn}, L2={use_l2}, sched={sched}")

                  # a) Instantiate with gradual unfreezing
                  params = {'gradual_unfreezing': True, 'unfreezing_rate': fixed_rate}
                  model = NeuralNetwork(
                      basemodel,
                      37,
                      train_loader,
                      val_loader,
                      test_loader,
                      unfreeze=1,
                      parameters=params
                  )
                  model.model.to(device)

                  # b) Optionally freeze BatchNorm layers
                  if freeze_bn:
                      for m in model.model.modules():
                          if isinstance(m, nn.BatchNorm2d):
                              m.eval()
                              for p in m.parameters():
                                  p.requires_grad = False

                  # c) Build optimizer (head @1e-3, backbone @1e-4), optional L2
                  wd = 1e-2 if use_l2 else 0.0
                  head_p = list(model.model.fc.parameters())
                  back_p = [
                      p for n,p in model.model.named_parameters()
                      if p.requires_grad and not n.startswith("fc")
                  ]
                  model.optimizer = optim.AdamW([
                      {'params': head_p, 'lr':1e-3, 'weight_decay':wd},
                      {'params': back_p, 'lr':1e-4, 'weight_decay':wd},
                  ])

                  # d) Attach scheduler if desired
                  if sched == 'steplr':
                      model.scheduler = optim.lr_scheduler.StepLR(
                          model.optimizer,
                          step_size=5,
                          gamma=0.1
                      )
                  elif sched == 'onecycle':
                      total_steps = len(train_loader) * 10
                      model.scheduler = optim.lr_scheduler.OneCycleLR(
                          model.optimizer,
                          max_lr=[1e-3,1e-4],
                          total_steps=total_steps,
                          pct_start=0.3,
                          anneal_strategy='cos'
                      )
                  else:
                      model.scheduler = None

                  # e) Train & record
                  acc = model.train2(epochs=15)
                  results.append((use_aug, freeze_bn, use_l2, sched, acc))

# Summary for comparison
print("\n=== Strategy 2 Grid Search Results ===")
print("aug | freeze_bn | L2   | scheduler | TestAcc")
print("----|-----------|------|-----------|--------")
for aug, fb, l2, sch, acc in results:
    print(f"{str(aug):<3} | {str(fb):<9} | {str(l2):<4} | {sch:<9} | {acc:6.2f}%")



>>> aug=True, freeze_bn=False, L2=False, sched=none


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch  1 | Train: 58.21% | Val: 86.14%
Epoch  2 | Train: 86.71% | Val: 88.04%
Epoch  3 | Train: 89.61% | Val: 89.40%
Epoch  4 | Train: 91.12% | Val: 88.04%
Epoch  5 | Train: 92.36% | Val: 88.86%
Epoch  6 | Train: 93.09% | Val: 89.40%
Epoch  7 | Train: 93.33% | Val: 87.50%
Epoch  8 | Train: 94.05% | Val: 88.86%
Epoch  9 | Train: 94.84% | Val: 89.13%
Epoch 10 | Train: 94.47% | Val: 89.67%
Epoch 11 | Train: 94.78% | Val: 87.77%
Epoch 12 | Train: 95.59% | Val: 87.50%
Epoch 13 | Train: 95.80% | Val: 88.59%
Epoch 14 | Train: 95.83% | Val: 88.59%
Epoch 15 | Train: 95.98% | Val: 87.50%
Test Accuracy: 88.28%

>>> aug=True, freeze_bn=False, L2=False, sched=none
Epoch  1 | Train: 58.09% | Val: 83.42%
Epoch  2 | Train: 85.60% | Val: 83.97%
Epoch  3 | Train: 88.71% | Val: 85.05%
Epoch  4 | Train: 89.89% | Val: 89.40%
Epoch  5 | Train: 91.73% | Val: 87.23%
Epoch  6 | Train: 91.52% | Val: 88.86%
Epoch  7 | Train: 92.57% | Val: 88.59%
Epoch  8 | Train: 93.72% | Val: 88.59%
Epoch  9 | Train: 94.41% | V

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f11645c4540>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f11645c4540>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 16

Epoch  3 | Train: 86.65% | Val: 84.51%


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f11645c4540>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f11645c4540>
self._shutdown_workers()Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
        Exception ignored in: self._shutdown_workers()if w.is_alive():
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f11645c4540>
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers

  Traceback (most recent call last):
      File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    s

Epoch  4 | Train: 87.29% | Val: 82.07%


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f11645c4540>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
   Exception ignored in:  <function _MultiProcessingDataLoaderIter.__del__ at 0x7f11645c4540> 
 Traceback (most recent call last):
   File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
^    ^self._shutdown_workers()^
^^^^^  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
^    if w.is_alive():^Exception ignored in: ^<function _MultiProcessingDataLoaderIter.__del__ at 0x7f11645c4540>

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader

Epoch  5 | Train: 88.47% | Val: 86.14%
Epoch  6 | Train: 88.38% | Val: 83.70%
Epoch  7 | Train: 89.07% | Val: 85.05%
Epoch  8 | Train: 90.52% | Val: 85.87%
Epoch  9 | Train: 90.55% | Val: 85.33%
Epoch 10 | Train: 91.12% | Val: 85.05%
Epoch 11 | Train: 91.27% | Val: 85.05%
Epoch 12 | Train: 92.12% | Val: 85.87%
Epoch 13 | Train: 91.30% | Val: 84.78%
Epoch 14 | Train: 91.79% | Val: 84.51%
Epoch 15 | Train: 92.57% | Val: 86.96%
Test Accuracy: 88.03%

>>> aug=True, freeze_bn=False, L2=False, sched=none
Epoch  1 | Train: 60.14% | Val: 85.60%
Epoch  2 | Train: 88.07% | Val: 88.59%
Epoch  3 | Train: 91.21% | Val: 89.40%
Epoch  4 | Train: 92.57% | Val: 88.86%
Epoch  5 | Train: 93.42% | Val: 89.13%
Epoch  6 | Train: 93.57% | Val: 89.95%
Epoch  7 | Train: 95.02% | Val: 91.03%
Epoch  8 | Train: 96.14% | Val: 88.32%
Epoch  9 | Train: 95.86% | Val: 90.76%
Epoch 10 | Train: 96.53% | Val: 90.49%
Epoch 11 | Train: 96.35% | Val: 89.95%
Epoch 12 | Train: 96.35% | Val: 89.40%
Epoch 13 | Train: 97.07% | V

## Imbalanced classes

1. Load the annotations/trainval.txt file

In [ ]:
# 0) Prepare names and label maps
names = []
breeds_map = {}

with open("data/oxford-iiit-pet/annotations/trainval.txt", "r") as f:
    for line in f:
        parts = line.strip().split()
        name = parts[0]
        class_id = int(parts[1]) - 1  # 0-indexed
        names.append(name)
        breeds_map[name] = class_id

breed_to_id = {i: i for i in set(breeds_map.values())}

Training with normal cross-entropy loss with limited data

In [ ]:
## Train only using 20%
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torchvision.datasets import OxfordIIITPet
from torch.utils.data import (
    Dataset,
    DataLoader,
    SubsetRandomSampler,
    WeightedRandomSampler
)

# --- Ensure device is defined ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load trainval metadata
tmp_train = OxfordIIITPet(root="data", split="trainval", target_types="category", download=True)
# Extract image stems: e.g. "Abyssinian_1"
names = [p.stem for p in tmp_train._images]

# Build mapping from stem → breed_name
breeds_map = {n: n.rsplit("_", 1)[0] for n in names}
breed_names = sorted(set(breeds_map.values()))   # 37 unique breed strings
breed_to_id = {b: i for i, b in enumerate(breed_names)}

class MultiClassPet(Dataset):
    def __init__(self, root, names, breeds_map, transform=None):
        self.root = root
        self.names = names
        self.breeds_map = breeds_map
        self.transform = transform

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        nm = self.names[idx]
        img_path = os.path.join(self.root, "oxford-iiit-pet", "images", nm + ".jpg")
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = breed_to_id[self.breeds_map[nm]]
        return img, label

# Transforms
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
root = "data"
# Full train/val dataset
full_dataset = MultiClassPet(root, names, breeds_map, transform=transform)

# Build 20%-per-class imbalanced index list
class_idxs = defaultdict(list)
for idx, nm in enumerate(names):
    lbl = breed_to_id[breeds_map[nm]]
    class_idxs[lbl].append(idx)

# Create an artificially imbalanced dataset with some breeds at 20%
imbalanced = []
class_percentages = {}  # To track how many samples are used from each class

for lbl, idxs in class_idxs.items():
    # Use 20% of data for each breed
    k = max(1, int(0.2 * len(idxs)))
    selected_idxs = random.sample(idxs, k)
    imbalanced += selected_idxs
    class_percentages[breed_names[lbl]] = (k, len(idxs), k/len(idxs)*100)

# Print dataset stats
print(f"Total samples in imbalanced dataset: {len(imbalanced)}")
print("Class distribution (samples, total, percentage):")
for breed, (count, total, percentage) in sorted(class_percentages.items()):
    print(f"{breed}: {count}/{total} ({percentage:.1f}%)")

# Split imbalanced list into train/val (80/20)
random.shuffle(imbalanced)
split = int(0.8 * len(imbalanced))
train_idxs = imbalanced[:split]
val_idxs = imbalanced[split:]

batch_size = 32
train_loader = DataLoader(
    full_dataset,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_idxs),
    num_workers=4
)
val_loader = DataLoader(
    full_dataset,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_idxs),
    num_workers=4
)

# Test loader via torchvision to avoid missing stems
test_ds = OxfordIIITPet(
    root=root,
    split="test",
    target_types="category",
    transform=transform,
    download=False
)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4)

# Define the NeuralNetwork class for training and evaluation
def make_model(unfreeze_layers=5):
    """
    Returns a NeuralNetwork instance fine-tuned on the
    imbalanced train_loader/val_loader/test_loader.
    """
    basemodel = models.resnet18(weights='IMAGENET1K_V1')

    m = NeuralNetwork(
        basemodel,
        len(breed_names),
        train_loader,
        val_loader,
        test_loader,
        unfreeze=unfreeze_layers
    )
    return m

# Train baseline model with normal cross-entropy loss
print("\n==== Training Baseline Model with Standard Cross-Entropy Loss ====")
model_base = make_model(unfreeze_layers=5)
base_acc = model_base.train(epochs=10)
print(f"⮞ Baseline (last 5 layers) Test Acc: {base_acc:.2f}%")

# Analyze the results - impact on the classes with limited data
print("\n==== Analysis of Impact on Classes with Limited Data ====")
test_acc, per_class_acc, class_counts = model_base.evaluate()

# Calculate correlation between class sample counts and accuracy
class_samples = [class_percentages[breed][0] for breed in breed_names]
correlation = np.corrcoef(class_samples, per_class_acc)[0, 1]
print(f"Correlation between class sample count and accuracy: {correlation:.4f}")

# Compare accuracy of classes with fewer samples vs classes with more samples
# (Since all classes are at 20%, this might not show significant differences)
print("\nAverage accuracy across all classes: {:.2f}%".format(sum(per_class_acc) / len(per_class_acc)))

Using device: cpu
Total samples in imbalanced dataset: 734
Class distribution (samples, total, percentage):
Abyssinian: 20/100 (20.0%)
Bengal: 20/100 (20.0%)
Birman: 20/100 (20.0%)
Bombay: 19/96 (19.8%)
British_Shorthair: 20/100 (20.0%)
Egyptian_Mau: 18/93 (19.4%)
Maine_Coon: 20/100 (20.0%)
Persian: 20/100 (20.0%)
Ragdoll: 20/100 (20.0%)
Russian_Blue: 20/100 (20.0%)
Siamese: 19/99 (19.2%)
Sphynx: 20/100 (20.0%)
american_bulldog: 20/100 (20.0%)
american_pit_bull_terrier: 20/100 (20.0%)
basset_hound: 20/100 (20.0%)
beagle: 20/100 (20.0%)
boxer: 20/100 (20.0%)
chihuahua: 20/100 (20.0%)
english_cocker_spaniel: 19/96 (19.8%)
english_setter: 20/100 (20.0%)
german_shorthaired: 20/100 (20.0%)
great_pyrenees: 20/100 (20.0%)
havanese: 20/100 (20.0%)
japanese_chin: 20/100 (20.0%)
keeshond: 20/100 (20.0%)
leonberger: 20/100 (20.0%)
miniature_pinscher: 20/100 (20.0%)
newfoundland: 19/96 (19.8%)
pomeranian: 20/100 (20.0%)
pug: 20/100 (20.0%)
saint_bernard: 20/100 (20.0%)
samoyed: 20/100 (20.0%)
scot

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/youngbinpyo/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/youngbinpyo/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/youngbinpyo/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^    ^self = reduction.pickle.load(from_parent)^
^^^^^^
  File "/Users/youngbinpyo/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'MultiClassPet' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^

RuntimeError: DataLoader worker (pid(s) 2885) exited unexpectedly

Strategy using weighted cross-entropy and over-sampling of the minority classes to compensate for the imbalanced training set

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torchvision.datasets import OxfordIIITPet
from torch.utils.data import (
    Dataset,
    DataLoader,
    SubsetRandomSampler,
    WeightedRandomSampler
)

# --- Ensure device is defined ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load trainval metadata
tmp_train = OxfordIIITPeFalse | True      | True | steplr    |  90.24%t(root="data", split="trainval", target_types="category", download=True)
# Extract image stems: e.g. "Abyssinian_1"
names = [p.stem for p in tmp_train._images]

# Build mapping from stem → breed_name
breeds_map = {n: n.rsplit("_", 1)[0] for n in names}
breed_names = sorted(set(breeds_map.values()))   # 37 unique breed strings
breed_to_id = {b: i for i, b in enumerate(breed_names)}

class MultiClassPet(Dataset):
    def __init__(self, root, names, breeds_map, transform=None):
        self.root = root
        self.names = names
        self.breeds_map = breeds_map
        self.transform = transform

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        nm = self.names[idx]
        img_path = os.path.join(self.root, "oxford-iiit-pet", "images", nm + ".jpg")
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = breed_to_id[self.breeds_map[nm]]
        return img, label

# Transforms
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
root = "data"
# Full train/val dataset
full_dataset = MultiClassPet(root, names, breeds_map, transform=transform)

# Build 20%-per-class imbalanced index list
class_idxs = defaultdict(list)
for idx, nm in enumerate(names):
    lbl = breed_to_id[breeds_map[nm]]
    class_idxs[lbl].append(idx)

# Create an artificially imbalanced dataset with some breeds at 20%
imbalanced = []
class_percentages = {}  # To track how many samples are used from each class

for lbl, idxs in class_idxs.items():
    # Use 20% of data for each breed
    k = max(1, int(0.2 * len(idxs)))
    selected_idxs = random.sample(idxs, k)
    imbalanced += selected_idxs
    class_percentages[breed_names[lbl]] = (k, len(idxs), k/len(idxs)*100)

# Print dataset stats
print(f"Total samples in imbalanced dataset: {len(imbalanced)}")
print("Class distribution (samples, total, percentage):")
for breed, (count, total, percentage) in sorted(class_percentages.items()):
    print(f"{breed}: {count}/{total} ({percentage:.1f}%)")

# Split imbalanced list into train/val (80/20)
random.shuffle(imbalanced)
split = int(0.8 * len(imbalanced))
train_idxs = imbalanced[:split]
val_idxs = imbalanced[split:]

batch_size = 32
train_loader = DataLoader(
    full_dataset,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_idxs),
    num_workers=4
)
val_loader = DataLoader(
    full_dataset,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_idxs),
    num_workers=4
)

# Test loader via torchvision to avoid missing stems
test_ds = OxfordIIITPet(
    root=root,
    split="test",
    target_types="category",
    transform=transform,
    download=False
)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4)

# Define the NeuralNetwork class for training and evaluation
def make_model(unfreeze_layers=5):
    """
    Returns a NeuralNetwork instance fine-tuned on the
    imbalanced train_loader/val_loader/test_loader.
    """
    basemodel = models.resnet18(weights='IMAGENET1K_V1')

    m = NeuralNetwork(
        basemodel,
        len(breed_names),
        train_loader,
        val_loader,
        test_loader,
        unfreeze=unfreeze_layers
    )
    return m

# Train baseline model with normal cross-entropy loss
print("\n==== Training Baseline Model with Standard Cross-Entropy Loss ====")
model_base = make_model(unfreeze_layers=5)
base_acc = model_base.train(epochs=10)
print(f"⮞ Baseline (last 5 layers) Test Acc: {base_acc:.2f}%")

# Analyze the results - impact on the classes with limited data
print("\n==== Analysis of Impact on Classes with Limited Data ====")
test_acc, per_class_acc, class_counts = model_base.evaluate()
baseline_per_class_acc = per_class_acc.copy()  # Store for comparison

# Calculate correlation between class sample counts and accuracy
class_samples = [class_percentages[breed][0] for breed in breed_names]
correlation = np.corrcoef(class_samples, per_class_acc)[0, 1]
print(f"Correlation between class sample count and accuracy: {correlation:.4f}")

# Compare accuracy of classes with fewer samples vs classes with more samples
# (Since all classes are at 20%, this might not show significant differences)
print("\nAverage accuracy across all classes: {:.2f}%".format(sum(per_class_acc) / len(per_class_acc)))

# ===== WEIGHTED CROSS-ENTROPY APPROACH =====
print("\n==== Training with Weighted Cross-Entropy Loss ====")

# Compute class weights inversely proportional to class frequencies
# First, count samples per class in training set
per_labels = [breed_to_id[breeds_map[names[i]]] for i in train_idxs]
class_counts = torch.tensor([per_labels.count(c) for c in range(len(breed_names))], dtype=torch.float)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum() * len(class_counts)  # Normalize
class_weights = class_weights.to(device)

print("Class weights:")
for i, (breed, weight) in enumerate(zip(breed_names, class_weights.cpu().numpy())):
    samples = per_labels.count(i)
    print(f"{breed}: {weight:.4f} (samples: {samples})")

# Train with weighted cross-entropy
model_w = make_model(unfreeze_layers=5)
model_w.criterion = nn.CrossEntropyLoss(weight=class_weights)
model_w.optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model_w.model.parameters()),
    lr=1e-3,
    weight_decay=1e-4
)
w_acc = model_w.train(epochs=10)
print(f"⮞ Weighted CE (last 5 layers) Test Acc: {w_acc:.2f}%")

# Analyze per-class improvements
_, w_per_class_acc, _ = model_w.evaluate()
print("\nPer-class accuracy changes (Weighted CE vs Baseline):")
for i, (breed, base_acc, w_acc) in enumerate(zip(breed_names, baseline_per_class_acc, w_per_class_acc)):
    diff = w_acc - base_acc
    print(f"{breed}: {base_acc:.2f}% → {w_acc:.2f}% ({diff:+.2f}%)")

# ===== OVERSAMPLING APPROACH =====
print("\n==== Training with Oversampling of Minority Classes ====")

# Implement weighted random sampling to oversample minority classes
sample_weights = [1.0 / class_counts[per_labels[i]].item() for i in range(len(train_idxs))]
oversampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights) * 2, replacement=True)

# Create a new dataloader with oversampling
oversampled_train_loader = DataLoader(
    full_dataset,
    batch_size=batch_size,
    sampler=oversampler,
    num_workers=4
)

# Train with oversampling
model_o = make_model(unfreeze_layers=5)
model_o.train_loader = oversampled_train_loader
model_o.optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model_o.model.parameters()),
    lr=1e-3,
    weight_decay=1e-4
)
o_acc = model_o.train(epochs=10)
print(f"⮞ Oversampling (last 5 layers) Test Acc: {o_acc:.2f}%")

# Analyze per-class improvements
_, o_per_class_acc, _ = model_o.evaluate()
print("\nPer-class accuracy changes (Oversampling vs Baseline):")
for i, (breed, base_acc, o_acc) in enumerate(zip(breed_names, baseline_per_class_acc, o_per_class_acc)):
    diff = o_acc - base_acc
    print(f"{breed}: {base_acc:.2f}% → {o_acc:.2f}% ({diff:+.2f}%)")

# ===== COMPARE ALL APPROACHES =====
print("\n==== Overall Comparison of Approaches ====")
print(f"Baseline Test Accuracy: {base_acc:.2f}%")
print(f"Weighted CE Test Accuracy: {w_acc:.2f}%")
print(f"Oversampling Test Accuracy: {o_acc:.2f}%")

# Plot comparison of per-class accuracies across methods
plt.figure(figsize=(15, 10))
x = np.arange(len(breed_names))
width = 0.25

plt.bar(x - width, baseline_per_class_acc, width, label='Baseline')
plt.bar(x, w_per_class_acc, width, label='Weighted CE')
plt.bar(x + width, o_per_class_acc, width, label='Oversampling')

plt.xlabel('Breed')
plt.ylabel('Accuracy (%)')
plt.title('Per-class Accuracy Comparison Across Methods')
plt.xticks(x, breed_names, rotation=90)
plt.legend()
plt.tight_layout()
plt.savefig('comparison_per_class.png')
plt.close()

# Calculate average improvement across all classes
w_avg_improvement = sum(w - b for w, b in zip(w_per_class_acc, baseline_per_class_acc)) / len(breed_names)
o_avg_improvement = sum(o - b for o, b in zip(o_per_class_acc, baseline_per_class_acc)) / len(breed_names)

print(f"\nAverage per-class improvement with Weighted CE: {w_avg_improvement:.2f}%")
print(f"Average per-class improvement with Oversampling: {o_avg_improvement:.2f}%")

# Extension

## Catastrophic Forgetting

In [ ]:
class Flowers102Dataset(Dataset):
    def __init__(self, root, split='train', transform=None, download=False):
        self.dataset = Flowers102(root=root, split=split, transform=transform, download=download)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return image, label

In [ ]:
train_ds = Flowers102Dataset("data", split='train', download=True, transform=transformIMG)
val_ds = Flowers102Dataset("data", split='val', download=True, transform=transformIMG)
test_ds = Flowers102Dataset("data", split='test', download=True, transform=transformIMG)

100%|██████████| 345M/345M [00:18<00:00, 19.1MB/s]
100%|██████████| 502/502 [00:00<00:00, 1.19MB/s]
100%|██████████| 15.0k/15.0k [00:00<00:00, 32.8MB/s]


In [ ]:
batch_size = 32

In [ ]:

def filter_first_n_classes(dataset, n_classes):

    indices = [i for i, (_, label) in enumerate(dataset) if label < n_classes]
    return Subset(dataset, indices)

In [ ]:
from torch.utils.data import Subset

# Filter to only first 37 classes
train_ds_flower = filter_first_n_classes(train_ds, 37)
val_ds_flower = filter_first_n_classes(val_ds, 37)
test_ds_flower = filter_first_n_classes(test_ds, 37)

In [ ]:
#False | True      | True | steplr    |  90.24%
use_aug = False
freeze_bn    = True
use_l2       = True
sched    = 'steplr'
fixed_rate = 2.0
breed_names = sorted(set(breeds_map.values()))

In [ ]:
annnotations_dir = os.path.join("data", "oxford-iiit-pet", "annotations")
lines_for_files = open(os.path.join(annnotations_dir, "list.txt")).read().splitlines()[6:]
breeds_map = {l.split()[0]:re.split(r'_\d+', l.split()[0])[0] for l in lines_for_files}
breed_to_id = {val: i for i, val in enumerate(sorted(set(breeds_map.values())))}
id_to_breed = {v: k for k, v in breed_to_id.items()}

class MultiClassPet(Dataset):
    def __init__(self, root, names, breeds_map, transform=None):
        self.root = root
        self.names = names
        self.breeds_map = breeds_map
        self.transform = transform

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        name = self.names[idx]
        img_path = os.path.join(
            self.root, "oxford-iiit-pet", "images", name + ".jpg"
        )
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = breed_to_id[self.breeds_map[name]]
        return img, label

full = MultiClassPet("data", trainval_names, breeds_map, transform=transformIMG)
n_val = int(0.1 * len(full))
train_ds, val_ds = random_split(full, [len(full)-n_val, n_val])
test_ds = MultiClassPet("data", test_names, breeds_map, transform=transformIMG)

batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Train best model
print(f"\n>>> aug={use_aug}, freeze_bn={freeze_bn}, L2={use_l2}, sched={sched}")

# a) Instantiate with gradual unfreezing
params = {'gradual_unfreezing': False, 'unfreezing_rate': fixed_rate}
model = NeuralNetwork(
    basemodel,
    len(breed_names),
    train_loader,
    val_loader,
    test_loader,
    unfreeze=6,
    parameters=params
)
model.model.to(device)

# b) Optionally freeze BatchNorm layers
if freeze_bn:
    for m in model.model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.eval()
            for p in m.parameters():
                p.requires_grad = False

# c) Build optimizer (head @1e-3, backbone @1e-4), optional L2
wd = 1e-2 if use_l2 else 0.0
head_p = list(model.model.fc.parameters())
back_p = [
    p for n,p in model.model.named_parameters()
    if p.requires_grad and not n.startswith("fc")
]
model.optimizer = optim.AdamW([
    {'params': head_p, 'lr':1e-3, 'weight_decay':wd},
    {'params': back_p, 'lr':1e-4, 'weight_decay':wd},
])

# d) Attach scheduler if desired
if sched == 'steplr':
    model.scheduler = optim.lr_scheduler.StepLR(
        model.optimizer,
        step_size=5,
        gamma=0.1
    )
elif sched == 'onecycle':
    total_steps = len(train_loader) * 10
    model.scheduler = optim.lr_scheduler.OneCycleLR(
        model.optimizer,
        max_lr=[1e-3,1e-4],
        total_steps=total_steps,
        pct_start=0.3,
        anneal_strategy='cos'
    )
else:
    model.scheduler = None

# e) Train & record
acc = model.train_catastrophic(epochs=15, tee=False)
print(acc)
bestmodel = model


>>> aug=False, freeze_bn=True, L2=True, sched=steplr


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch  1 | Train: 73.82% | Val: 89.67%
Epoch  2 | Train: 94.17% | Val: 89.67%
Epoch  3 | Train: 97.86% | Val: 91.30%
Epoch  4 | Train: 99.21% | Val: 91.30%
Epoch  5 | Train: 99.64% | Val: 91.03%
Epoch  6 | Train: 99.79% | Val: 90.49%
Epoch  7 | Train: 99.91% | Val: 91.58%
Epoch  8 | Train: 99.97% | Val: 90.22%
Epoch  9 | Train: 99.97% | Val: 91.30%
Epoch 10 | Train: 100.00% | Val: 91.58%
Epoch 11 | Train: 100.00% | Val: 91.03%
Epoch 12 | Train: 99.97% | Val: 91.03%
Epoch 13 | Train: 99.94% | Val: 90.49%
Epoch 14 | Train: 99.97% | Val: 91.30%
Epoch 15 | Train: 99.97% | Val: 90.49%
{'primary': [], 'secondary': []}


In [ ]:
train_loader_flower = DataLoader(train_ds_flower, batch_size=batch_size, shuffle=True,  num_workers=4, pin_memory=True)
val_loader_flower   = DataLoader(val_ds_flower,   batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader_flower  = DataLoader(test_ds_flower,  batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
back = copy.deepcopy(model)

In [ ]:
model.train_loader = train_loader_flower
model.val_loader = val_loader_flower
model.test_loader = test_loader_flower

In [ ]:
model.optimizer = optim.AdamW([
    {'params': head_p, 'lr':1e-1, 'weight_decay':wd},
    {'params': back_p, 'lr':1e-2, 'weight_decay':wd},
])
model.scheduler = None
model.train_catastrophic(epochs=10, test_loader=test_loader)
model.optimizer = optim.AdamW([
    {'params': head_p, 'lr':1e-3, 'weight_decay':wd},
    {'params': back_p, 'lr':1e-4, 'weight_decay':wd},
])
model.scheduler = optim.lr_scheduler.StepLR(
        model.optimizer,
        step_size=5,
        gamma=0.1
    )
model.train_catastrophic(epochs=20, test_loader=test_loader)


Epoch  1 | Train: 10.81% | Val: 8.11%
Test Accuracy: 11.37%
Test Accuracy: 4.09%
Epoch  2 | Train: 30.27% | Val: 35.14%
Test Accuracy: 26.65%
Test Accuracy: 4.50%
Epoch  3 | Train: 41.89% | Val: 34.59%
Test Accuracy: 36.18%
Test Accuracy: 4.12%
Epoch  4 | Train: 52.16% | Val: 57.30%
Test Accuracy: 61.83%
Test Accuracy: 4.06%
Epoch  5 | Train: 74.86% | Val: 74.86%
Test Accuracy: 70.36%
Test Accuracy: 4.72%
Epoch  6 | Train: 84.32% | Val: 72.16%
Test Accuracy: 73.06%
Test Accuracy: 4.28%
Epoch  7 | Train: 90.00% | Val: 73.24%


KeyboardInterrupt: 

In [ ]:
model.train_loader = train_loader
model.val_loader = val_loader
model.test_loader = test_loader

In [ ]:
model.test_model()